In [12]:
import pandas as pd
import numpy as np
import requests
import json
import sys
import os
import pickle
import xgboost as xgb 
print(xgb.__version__)

1.5.0


In [2]:
from config_data import df, api, \
                        comet_config, \
                        filename_dict, \
                        features_dict

In [3]:
# Test '/predict'
predicted_probs = requests.post(
    "http://127.0.0.1:5000/predict", 
    json=json.loads(df.iloc[0:5].to_json())
)
print(predicted_probs.json())

{'response': [[0.9221800135286895, 0.0778199864713105], [0.8687570775153671, 0.1312429224846329], [0.8664541598372335, 0.13354584016276647], [0.8608866225555861, 0.1391133774444139], [0.9199142931355345, 0.0800857068644655]]}


## XGBoost Q5-2 Hyperparameter Tuning

In [26]:
model1 = 'xgboost-5-2'
version1 = '1.0.3'

request1 = requests.post(
    "http://127.0.0.1:5000/download_registry_model", 
    json = {'workspace': 'xiaoxin-zhou', 'model': model1, 'version': version1}
)

```
...
[2021-12-17 16:40:35,680] INFO in app: ---------------------running download_registry_model()---------------------
[2021-12-17 16:40:35,681] INFO in app: {'workspace': 'xiaoxin-zhou', 'model': 'xgboost-5-2', 'version': '1.0.3'}
[2021-12-17 16:40:35,682] INFO in app: Model q5_2_tuned.json is already downloaded, loading ...
[2021-12-17 16:40:35,771] INFO in app: Succesfully loaded default model q5_2_tuned.json
```

In [27]:
request1 = requests.post(
    "http://127.0.0.1:5000/predict", 
    json=json.loads(df.iloc[0:5].to_json())
)
import pprint
pprint.pprint(request1.json())

{'response': [[0.9991273283958435, 0.0008726531523279846],
              [0.72276371717453, 0.27723628282546997],
              [0.9218897819519043, 0.0781102254986763],
              [0.9532418251037598, 0.04675816744565964],
              [0.9751328229904175, 0.02486715465784073]]}


## XGBoost Q5-3 VarianceThreshold

In [70]:
model2 = 'xgboost-model-var-threshold'
version2 = '1.0.1'

request2 = requests.post(
    "http://127.0.0.1:5000/download_registry_model", 
    json = {'workspace': 'xiaoxin-zhou', 'model': model2, 'version': version2}
)



```
...
[2021-12-17 16:47:34,128] INFO in app: ---------------------running download_registry_model()---------------------
[2021-12-17 16:47:34,129] INFO in app: {'workspace': 'xiaoxin-zhou', 'model': 'xgboost-model-var-threshold', 'version': '1.0.1'}
[2021-12-17 16:47:34,130] INFO in app: q5_3_var_threshold.json doesn't exist, downloading ...
COMET INFO: Downloading registry model 'xgboost-model-var-threshold', version '1.0.1', stage None from workspace 'xiaoxin-zhou'...
COMET INFO: Unzipping model to '/Users/xiaoxinzhou/Documents/2021-Fall/UdeM/IFT6758/IFT6758-docker-project' ...
COMET INFO: done!
[2021-12-17 16:47:37,782] INFO in app: Succesfully downloaded model name: q5_3_var_threshold.json
[2021-12-17 16:47:37,828] INFO in app: Succesfully loaded default model q5_3_var_threshold.json
```

In [71]:
request2 = requests.post(
    "http://127.0.0.1:5000/predict", 
    json=json.loads(df.iloc[0:5].to_json())
)
import pprint
pprint.pprint(request2.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [37]:
f0 = 'ift6758/ift6758/data/q5_2_tuned.json'
f1 = 'ift6758/ift6758/data/q5_3_var_threshold.json'
m = xgb.XGBClassifier()
m.load_model(f0)

In [38]:
m.load_model(f1)

XGBoostError: [17:13:33] /Users/runner/miniforge3/conda-bld/xgboost-split_1634712680264/work/src/common/io.cc:102: Opening ift6758/ift6758/data/q5_3_var_threshold.json failed: No such file or directory
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000016c147c74 dmlc::LogMessageFatal::~LogMessageFatal() + 116
  [bt] (1) 2   libxgboost.dylib                    0x000000016c16e6b8 xgboost::common::LoadSequentialFile(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, bool) + 840
  [bt] (2) 3   libxgboost.dylib                    0x000000016c151167 XGBoosterLoadModel + 551
  [bt] (3) 4   libffi.8.dylib                      0x000000010bd87d92 ffi_call_unix64 + 82
  [bt] (4) 5   ???                                 0x0000000305a52d40 0x0 + 12979612992



In [39]:
# Get XGBoost Classifier feature names
clf = m.get_booster()
feature_names = clf.feature_names

In [41]:
feature_names

['eventIdx',
 'game_id',
 'Game Seconds',
 'Game Period',
 'X-Coordinate',
 'Y-Coordinate',
 'Shot Distance',
 'Shot Angle',
 'Shot Type',
 'Was Net Empty',
 'Last Event Type',
 'Last X-Coordinate',
 'Last Y-Coordinate',
 'Time from Last Event (seconds)',
 'Distance from Last Event',
 'Is Rebound',
 'Change in Shot Angle',
 'Speed']

In [48]:
df[feature_names].shape

(282671, 18)